# The `py_aff3ct` 'Hello World'

This notebook presents the first steps for playing with [AFF3CT](https://aff3ct.github.io/). 

## Objectives

In this notebook, we cover the following key concepts of `py_aff3ct`:
* `module`
* `task`
* `socket`

At the end of this notebook, you should be able to :
* Create a `module`
* Get information about the `task` of a `module`
* Bind two `sockets`
* Execute a `task` of a `module`
* Read the values of a `socket`
* Activate the debug mode of a task

## Libraries import

Before going any further, we should import the `py_aff3ct` library. The `sys` library is required to add the library path to your path. For this example, we will also need the `numpy` library.

In [11]:
import sys  
sys.path.insert(0, '../../build/lib')
import numpy as np

import py_aff3ct

## Module building

The `py_aff3ct` library is composed of `modules`. Basically, a module is a device that can process data. A data processing is called a `task` in the AFF3CT language. Tasks exchange data through `sockets`.

Let's start by building a source `module` that generates random binary data of length 8.

In [12]:
src = py_aff3ct.module.source.Source_random(8)

 This module has a task called `generate`. Let us display information about this `task`.
 Now we can generate the bits randomly by executing the task named `generate`.

In [13]:
src('generate').info()

The use of parenthesis for accessing tasks is deprecated, use brackets instead.
- Task 
	- Name         : generate
	- Sockets      : 
	- Socket 0
		- Name              : U_K
		- Type              : out
		- Elements per frame: 8
		- Data type         : int32
	- Socket 1
		- Name              : status
		- Type              : out
		- Elements per frame: 1
		- Data type         : int32



## Module  execution

Let us execute the task `generate`

In [14]:
src('generate').exec()

The use of parenthesis for accessing tasks is deprecated, use brackets instead.


The generated bits are contained in the socket `U_K` and can be accessed from your python code as a NumPy array using the `module['task_name::socket_name']` syntax. 

In [15]:
bits = src['generate::U_K'][:]
print("bits = ", bits)

bits =  [[1 1 1 1 1 1 1 1]]


## Second module building

We now want to modulate the bits of `U_K` using BPSK modulation. So, we start by building a BPSK `modem` (which is a also a `module`).

In [16]:
mod = py_aff3ct.module.modem.Modem_BPSK(8)

When asking for more details about the task `modulate` of object `mod` we can see that this task has one input socket called `X_N1`.

In [17]:
mod('modulate').info()

The use of parenthesis for accessing tasks is deprecated, use brackets instead.
- Task 
	- Name         : modulate
	- Sockets      : 
	- Socket 0
		- Name              : X_N1
		- Type              : in
		- Elements per frame: 8
		- Data type         : int32
	- Socket 1
		- Name              : X_N2
		- Type              : out
		- Elements per frame: 8
		- Data type         : float32
	- Socket 2
		- Name              : status
		- Type              : out
		- Elements per frame: 1
		- Data type         : int32



## Module binding 

To link the source `src` and the modulator `mod`, the output socket `U_K` of the task `generate` of the module `src` should be "binded" to the input socket `X_N1` of the task `modulate` of the module `mod`.

In [18]:
mod['modulate::X_N1'] = src['generate::U_K']

## Tasks execution

Now we can execute the tasks of each module.

In [19]:
src['generate'].exec()
mod['modulate'].exec()
bits    = src['generate::U_K'][:]
symbols = mod['modulate::X_N2'][:]
print("bits    = ", bits)
print("symbols = ", symbols)

bits    =  [[0 1 1 0 0 0 1 0]]
symbols =  [[ 1. -1. -1.  1.  1.  1. -1.  1.]]


## Debug mode activation

You can see that the data is automatically transfered from the output of `generate` to the input of `modulate`. The data printing can be handled automatically by setting the `debug` property of the tasks to `true`.

In [20]:
src['generate'].debug = True
mod['modulate'].debug = True
src['generate'].exec()
mod['modulate'].exec()

# Source_random::generate(int32 U_K[8])
# {OUT} U_K = [    1,     0,     0,     0,     1,     1,     1,     0]
# Returned status: [0 'SUCCESS']
#
# Modem_BPSK::modulate(const int32 X_N1[8], float32 X_N2[8])
# {IN}  X_N1 = [    1,     0,     0,     0,     1,     1,     1,     0]
# {OUT} X_N2 = [-1.00,  1.00,  1.00,  1.00, -1.00, -1.00, -1.00,  1.00]
# Returned status: [0 'SUCCESS']
#
